<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
from importnb import Notebook
with Notebook(): 
    from che4 import Props

import numpy as np
from scipy.optimize import root

### NRTL Activity Coefficient Model

$ln (\gamma_i) = \dfrac{ \sum\limits_{j=1}^{n}x_j \tau_{ji}G_{ji}}{\sum\limits_{k=1}^{n}x_kG_{ki}} + 
\sum\limits_{j=1}^{n} \dfrac{x_jG_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}} \left( \tau_{ij} - 
\dfrac{\sum\limits_{m=1}^{n} x_m \tau_{mj} G_{mj}}{\sum\limits_{k=1}^{n} x_kG_{kj}} \right)$

$\tau_{ji} = A_{ji} + \dfrac{B_{ji}}{T} + C_{ji}ln(T)+D_{ji}T$

$G_{ji}=exp \left( - \alpha_{ji} \tau_{ji} \right)$

$\alpha_{ij}=\alpha_{ji}$

In [78]:
def lng_matvec(x,T):
    x=x.reshape((1,-1))
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = x@ (tau*G)
    xG = x@G
    xtauGdivxG=xtauG/xG
    a=G*(tau-xtauGdivxG[None,:])/xG[None,:]
    lng= xtauGdivxG + a@(x.squeeze())
    return lng.flatten()
np.exp(lng_matvec(x,300.))

array([0.99367274, 1.14457226, 1.61038962])

In [56]:
def lng_einsum(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = np.einsum('j,ji->i',x,tau*G)
    xG = np.einsum('j,ji',x,G)
    xtauGdivxG=xtauG/xG
    print(xtauGdivxG)
    term2=np.einsum('j,ij->i',x,G*(tau-xtauGdivxG[None,:])/xG[None,:] )
    return xtauGdivxG + term2


np.exp(lng_einsum(x,300.))

[ 0.49820792  0.19718116 -0.09022929]


array([0.99367274, 1.14457226, 1.61038962])

In [57]:
p = Props(['Methanol','Ethanol','Water'])

In [43]:
def lng_method1(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    lng = np.zeros_like(x)
    xtauGdivxG = np.zeros_like(x)
    xG = np.zeros_like(x)
    term2 = np.zeros_like(x)
    N = x.size
    for i in range(N):
        xtauG = 0.
        for j in range(N):
            xtauG += x[j]*tau[j,i]*G[j,i]
            xG[i] += x[j]*G[j,i]
        xtauGdivxG[i] = xtauG/xG[i]
    for i in range(N):
        for j in range(N):
            term2[i]+= x[j]*G[i,j]/xG[j]*(tau[i,j] - xtauGdivxG[j])

    print(term2)
    return xtauGdivxG + term2

In [65]:
x=np.array([1/3, 1/3, 1/3])
np.exp(lng_method1(x,300.))


[-0.50455528 -0.06215016  0.56670544]


array([0.99367274, 1.14457226, 1.61038962])

In [86]:
def bubblePy(x, T):
    Pi=x*np.exp(lng_matvec(x,T))*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P 

def bubbleT(x, P):

    r



In [87]:
bubblePy(x,350.)

(DeviceArray(111634.03410608, dtype=float64),
 DeviceArray([0.48004229, 0.32823712, 0.1917206 ], dtype=float64))

In [31]:
a

array([[ 14,  38,  62],
       [ 38, 126, 214]])

In [28]:
a@(b.T)

array([[ 14,  38],
       [ 38, 126],
       [ 62, 214]])